In [1]:
import pandas as pd
import torch
import networkx as nx
import torch_geometric.datasets as dataset

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Types
from torch_geometric.datasets import GNNBenchmarkDataset
from networkx import Graph

In [3]:
myData = dataset.GNNBenchmarkDataset(
    root="data/homework02",
    name="CSL",
)

In [4]:
# Pega a lista de grafos como uma lista de GNNBenchmarkDataset
graphs = myData.process_CSL()
#graphs[0].edge_stores

In [5]:
graphs[0]

Data(edge_index=[2, 164], y=0, num_nodes=41)

In [6]:
# Recebe um [GNNBenchmarkDataset] e cria 
# e retorna um grafo NetworkX
def createGraph(data: GNNBenchmarkDataset) -> Graph:    
    G = nx.Graph()
    for index in range(data.num_nodes):
        G.add_node(index)
        
    u = data.edge_index[0]
    v = data.edge_index[1]
    for index in range(data.num_edges):
        G.add_edge(int(u[index]), int(v[index]))
        
    return G

In [2]:
from grakel.datasets import fetch_dataset
'''
if as_graphs:
        for i in range(1, len(Graphs)+1):
            Gs.append(Graph(Graphs[i], node_labels[i], edge_labels[i]))
    else:
        for i in range(1, len(Graphs)+1):
            Gs.append([Graphs[i], node_labels[i], edge_labels[i]])
'''
MUTAG = fetch_dataset("MUTAG", verbose=False)
G = MUTAG.data
y = MUTAG.target

In [11]:
print(type(G[0]))
print(type(G[0][0]))
for el in G[0][0]:
    print(el)

<class 'list'>
<class 'set'>
(3, 4)
(4, 3)
(5, 4)
(12, 13)
(5, 7)
(14, 13)
(8, 9)
(9, 8)
(9, 14)
(17, 15)
(10, 9)
(1, 6)
(13, 14)
(6, 5)
(15, 17)
(4, 5)
(14, 9)
(5, 6)
(9, 10)
(1, 2)
(10, 11)
(2, 1)
(11, 10)
(6, 1)
(15, 13)
(15, 16)
(13, 15)
(16, 15)
(3, 2)
(12, 11)
(4, 10)
(8, 7)
(10, 4)
(2, 3)
(11, 12)
(13, 12)
(7, 5)
(7, 8)


In [15]:
from grakel.kernels import WeisfeilerLehman, VertexHistogram
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
wl_kernel = WeisfeilerLehman(n_iter=5, normalize=False, base_graph_kernel=VertexHistogram)

In [21]:
myGraphs = []
for graph in graphs:
    myGraphs.append( createGraph(graph) )
#map(graphs, createGraph)

y = [ graph.y for graph in graphs ]

In [16]:
G_train, G_test, y_train, y_test = train_test_split(G, y, test_size=0.1, random_state=42)

In [25]:
K_train = wl_kernel.fit_transform(G_train)
K_test = wl_kernel.transform(G_test)

In [26]:
K_train

array([[1.        , 0.73435802, 0.73903538, ..., 0.63577527, 0.65695235,
        0.63745526],
       [0.73435802, 1.        , 0.72828268, ..., 0.61806172, 0.6444991 ,
        0.62113251],
       [0.73903538, 0.72828268, 1.        , ..., 0.88391632, 0.86030249,
        0.79138739],
       ...,
       [0.63577527, 0.61806172, 0.88391632, ..., 1.        , 0.88242643,
        0.83370093],
       [0.65695235, 0.6444991 , 0.86030249, ..., 0.88242643, 1.        ,
        0.85546372],
       [0.63745526, 0.62113251, 0.79138739, ..., 0.83370093, 0.85546372,
        1.        ]])

In [21]:
'''
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='precomputed', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
'''
clf = SVC(kernel='precomputed')
clf.fit(K_train, y_train)
y_pred = clf.predict(K_test)

In [22]:
print(y_test)
print(y_pred)

[ 1  1 -1  1  1  1  1 -1 -1 -1  1 -1  1 -1  1  1 -1  1  1]
[ 1  1 -1  1  1  1  1  1 -1 -1  1 -1  1  1 -1  1 -1  1  1]


In [9]:
# Cria um grafo
graph = createGraph(graphs[0])
#fig = nx.draw(graph, with_labels = True, pos=nx.spring_layout(graph))

In [32]:
# para cada grafo, gerar a lista 
def weisfeiler(graph: Graph, epoc: int = None, maxColor: int = None) -> list[int]:
    graph = graph.copy()
    
    if epoc == None:
        epoc = len(graph.nodes)
    
    if maxColor == None:
        maxColor = len(graph.nodes)*5
    
    for node in graph.nodes:
        graph.nodes[node]['color'] = 1

    #print(graph.nodes(data=True))

    countColors = []
    curColor = 2
    while curColor < maxColor and epoc > 0:
        epoc = epoc - 1

        neighborColors = {}
        for node in graph.nodes():
            #print(f"No nó {node}")
            print( len(graph.adj[node]) )
            neighbornString = ""
            for neighbor in graph.adj[node]:
                # print(graph.nodes[neighbor]['color'], end="")
                neighbornString = neighbornString + "-" + str(graph.nodes[neighbor]['color'])

            if neighborColors.get(neighbornString, None) == None:
                neighborColors[neighbornString] = True
                print(neighbornString)
        
        break
    
    return countColors

print(weisfeiler(graph))

4
-1-1-1-1
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
[]


In [8]:
# criar um dataset com a lista + classe

In [9]:
# treinar um modelo

In [10]:
# fazer testes do modelo